In [ ]:
import os
import cv2
import glob
import json
import keras
import shutil
import itertools
import tensorflow
import numpy as np   # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import seaborn as sns
# from glob import glob
from PIL import Image
import tensorflow as tf
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from distutils.dir_util import copy_tree
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization

%matplotlib inline

In [ ]:
# Get working directory
work_dir = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/'
os.listdir(work_dir)

In [ ]:
# Make a new directory for the corn dataset
!mkdir corn

!mkdir ./corn/train
!mkdir ./corn/valid

In [ ]:
# Create different directories (folders) for the corn classes
!mkdir ./corn/train/Corn_(maize)___Northern_Leaf_Blight
!mkdir ./corn/train/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
!mkdir ./corn/train/Corn_(maize)___Common_rust_
!mkdir ./corn/train/Corn_(maize)___healthy

!mkdir ./corn/valid/Corn_(maize)___Northern_Leaf_Blight
!mkdir ./corn/valid/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
!mkdir ./corn/valid/Corn_(maize)___Common_rust_
!mkdir ./corn/valid/Corn_(maize)___healthy

### Get the Training data

In [ ]:
# Get the training data for the Corn Northern Leaf Blight
dest_dir = "./corn/train/Corn_(maize)___Northern_Leaf_Blight"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___Northern_Leaf_Blight'):
    print(file)
    copy_tree(file, dest_dir)

In [ ]:
# Get the training data for the Cercospora Leaf spot Gray Leaf spot
dest_dir = "./corn/train/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'):
    print(file)
    copy_tree(file, dest_dir)

In [ ]:
# Get the training data for the Corn Common rust
dest_dir = "./corn/train/Corn_(maize)___Common_rust_"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___Common_rust_'):
    print(file)
    copy_tree(file, dest_dir)

In [ ]:
# Get the training data for the Corn Healthy
dest_dir = "./corn/train/Corn_(maize)___healthy"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy'):
    print(file)
    copy_tree(file, dest_dir)

### Get the Validation data

In [ ]:
# Get the Validation data for Corn Northern Leaf Blight
dest_dir = "./corn/valid/Corn_(maize)___Northern_Leaf_Blight"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Corn_(maize)___Northern_Leaf_Blight'):
    print(file)
    copy_tree(file, dest_dir)

In [ ]:
# Get the Validation data for the Cercospora Leaf spot Gray Leaf spot
dest_dir = "./corn/valid/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'):
    print(file)
    copy_tree(file, dest_dir)

In [ ]:
# Get the Validation data for the Corn Common rust
dest_dir = "./corn/valid/Corn_(maize)___Common_rust_"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Corn_(maize)___Common_rust_'):
    print(file)
    copy_tree(file, dest_dir)

In [ ]:
# Get the Validation data for the Corn Healthy
dest_dir = "./corn/valid/Corn_(maize)___healthy"
for file in glob.glob('/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Corn_(maize)___healthy'):
    print(file)
    copy_tree(file, dest_dir)

### EDA

- Get training dataset for EDA

In [ ]:
# get training data
dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/"

# Get Images and length of images
Blight = glob.glob(os.path.join(dir+"Corn_(maize)___Northern_Leaf_Blight/"+"*"))
Common_Rust = glob.glob(os.path.join(dir+"Corn_(maize)___Common_rust_/"+"*"))
Gray_Leaf_Spot = glob.glob(os.path.join(dir+"Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/"+"*"))
healthy = glob.glob(os.path.join(dir+"Corn_(maize)___healthy/"+"*"))

# Print the length of each training class
print("Images with Blight, Common_Rust and Gray Leaf Spot are \n"+str(len(Blight))+"  \t"+ 
      str(len(Common_Rust))+"\t"+str(len(Gray_Leaf_Spot)))
print("Healthy Images are \n" + str(len(healthy)))

In [ ]:
# check if there is class imbalance
classes = [len(Common_Rust), len(Gray_Leaf_Spot), len(Blight), len(healthy)]
labels = ('Corn_(maize)___Common_rust_', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
           'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy')

y_pos = range(len(classes))

fig = plt.figure(figsize=(18,8))
plt.barh(y_pos, classes, align='center', alpha=1)
plt.yticks(y_pos, labels)
plt.xlabel('Number of images')
plt.title('Corn(Maize) Disease Prediction')
plt.show()

### Taking only first 8 images for Visualization

In [ ]:
blight = Blight[:8]
health = healthy[:8]
rust = Common_Rust[:8]
grayspot = Gray_Leaf_Spot[:8]

In [ ]:
# function to display images
def showImages(images):
    plt.figure(figsize=(20,35))
    for i,k  in enumerate(images):
        image = Image.open(k)
        plt.subplot(4,4,i+1)
        plt.imshow(image)

### Images of Gray Spot Leaf

In [ ]:
print("Showing Images of the  Gray Spot Leaf")
showImages(grayspot)

### Images of Common rust Leaf

In [ ]:
print("Showing Images of the Common rust Leaf")
showImages(rust)

### Images of Blight Leaf

In [ ]:
print("Showing Images of the Blight Leaf")
showImages(blight)

### Healthy Leaf images

In [ ]:
print("Showing Images of the Healthy Leaf")
showImages(health)

### Models to explore for our use-case
- VGG-16
- VGG19
- Inception-V3
- AlexNet
- ResNet
- SqueezeNet

#### Get the train and valid with path

In [ ]:
# path to both the training and validation dataset
path = "./corn"
train_path = os.path.join(path, "train")
print(os.listdir(train_path))
print("*"*100)
valid_path = os.path.join(path, "valid")
print(os.listdir(valid_path))

- Using the glob function to access the folders.

In [ ]:
# Use the glob function to get access to the training tomato datasets
folders_train = glob.glob("./corn/train/*")
folders_train
len(folders_train)

In [ ]:
# Use the glob function to get access to the validation tomato datasets
folders_valid = glob.glob("./corn/valid/*")
folders_valid
len(folders_valid)

### Visual Geometry Group (VGG) net

- The original input image of dimension 256 x 256 was augmented to a dimension of `227 x 227` for the `AlexNet model` and `224 x224` for the `VGG16 net`.

- vgg16 input shape and weight

- weights: 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.

- input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with channels_last data format) or (3, 224, 224) (with channels_first data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.

In [ ]:
def model_compile(training_folders, SIZE):
    vg16 = VGG16(input_shape = SIZE + [3], weights="imagenet", include_top=False)
    for layer in vg16.layers:
        layer.trainable = False 
    
    # Flattening the output layer
    x = Flatten()(vg16.output)
    prediction = Dense(len(training_folders), activation="softmax")(x)
    modelvg = Model(inputs=vg16.input, outputs=prediction)
    
    opt = Adam(lr=0.0001)       
    modelvg.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=opt)
    return modelvg

In [ ]:
# VGG image size
SIZE = [224, 224]
# Compiling the model
modelvg = model_compile(folders_train, SIZE)

- `VGG16` model summary

In [ ]:
modelvg.summary()

#### Data Augmentation
- `Data augmentation` is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data. `Data augmentation` techniques such as `cropping`, `padding`, and `horizontal flipping` are commonly used to train large neural networks.

- `Image data augmentation` is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset. ... Image data augmentation is used to expand the training dataset in order to improve the performance and ability of the model to generalize.

- `Image augmentation` is a technique of altering the existing data to create some more data for the model training process. In other words, it is the process of artificially expanding the available dataset for training a deep learning model.

- The `steps_per_epoch` argument must specify the number of batches of samples comprising one epoch. For example, if your original dataset has 10,000 images and your batch size is 32, then a reasonable value for `steps_per_epoch` when fitting a model on the augmented data might be `ceil(10,000/32)`, or `313 batches`.

- The images in the dataset are not used directly. Instead, only `augmented images` are provided to the model. Because the `augmentations` are performed randomly, this allows both modified images and close facsimiles of the original images (e.g. almost no augmentation) to be generated and used during training.

In [ ]:
def Model_fit(train_path, valid_path):
    # Image data augmentation is only applied to the training dataset
    # Using the Image DataGenerator class
    train_dataugmen_vg16 = ImageDataGenerator(rescale=1./255,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True)

    # Image data augmentation is not to be applied to the validation/testing dataset
    test_dataugmen_vg16 = ImageDataGenerator(rescale=1./255)
    
    train_set_vg16 = train_dataugmen_vg16.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode="categorical")
    
    valid_set_vg16 = test_dataugmen_vg16.flow_from_directory(valid_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode="categorical")
    
    r_vg16 = modelvg.fit(train_set_vg16,
                       validation_data=valid_set_vg16,
                       epochs=20, 
#                         epochs = train_set_vg19.n // train_set_vg19.batch_size,
                       steps_per_epoch=len(train_set_vg16),
                       validation_steps=len(valid_set_vg16))
    return r_vg16

In [ ]:
r_vg16 = Model_fit(train_path, valid_path)

- Visualization for VGG19

In [ ]:
import matplotlib.pyplot as plt

accuracy = r_vg16.history['accuracy']
val_accuracy = r_vg16.history['val_accuracy']

loss = r_vg16.history['loss']
val_loss = r_vg16.history['val_loss']

epochs = range(len(accuracy))
plt.title("VGG16 training accuracy and validation accuracy")
plt.plot(epochs, accuracy, "b", label="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.legend()
plt.show()

plt.title("VGG16 training loss and validation loss")
plt.plot(epochs, loss, "b", label="trainning loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.legend()
plt.show()

In [ ]:
# !mkdir ./model_output
# model_pth = "./model_output"
# os.listdir(model_pth)

In [ ]:
# Save the vgg16 model
# modelvg.save(os.path.join(model_pth, "VGG16_CORN_MODEL.h5"))

In [ ]:
# Save the vgg16 model
modelvg.save("VGG16_CORN_MODEL.h5")

In [ ]:
import cv2
import tensorflow as tf
def prepare(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    new_array = cv2.resize(img_array, (224, 224))
    return new_array.reshape(-1, 224, 224, 3)

model = tf.keras.models.load_model("VGG16_CORN_MODEL.h5") # "./model_output/VGG16_CORN_MODEL.h5"

In [ ]:
# Make predictions using the test dataset
prediction = model.predict([prepare("../input/new-plant-diseases-dataset/test/test/CornCommonRust1.JPG")])
np.argmax(prediction)

In [ ]:
prediction = model.predict([prepare("../input/new-plant-diseases-dataset/test/test/CornCommonRust3.JPG")])
np.argmax(prediction)

In [ ]:
prediction = model.predict([prepare("/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/1157b4eb-dfb3-41b7-a083-77b0ca2e7634___RS_GLSp 4460_new30degFlipLR.JPG")])
np.argmax(prediction)

In [ ]:
prediction = model.predict([prepare("/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Corn_(maize)___healthy/44927b10-a45a-4c36-becb-2d410f06d77e___R.S_HL 7967 copy 2_flipLR.jpg")])
np.argmax(prediction)

- Save the vgg16 model as a tar.gz file

In [ ]:
n_out = "./"
os.listdir(n_out)

In [ ]:
import tarfile
import os.path


# !mkdir ./model_output
model_pth = "./model_output"
# os.listdir(model_pth)

output_dir = "./model_output"
# os.listdir(output_dir)

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

source_dir = model_pth
output_filename = os.path.join(model_pth, 'VGG16_CORN_MODEL' + '.tar.gz')
make_tarfile(output_filename, source_dir)